In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(f"Name: {gpu.name}, Type: {gpu.device_type}")
else:
    print("No GPU devices found.")


In [31]:
# import pandas as pd

# # Define the C code snippets
# code_snippet_1 = """
# #include <stdio.h>
# // This is a single-line comment
# /* This is a
#    multi-line comment */
# #define MAX(x, y) ((x) > (y) ? (x) : (y))

# int main() {
#     // Print the maximum of two numbers
#     int a = 10;
#     int b = 20;
#     printf("Maximum is %d\n", MAX(a, b));
#     return 0;
# }
# """

# code_snippet_2 = """
# float x = 3.14;
# int y = 42; 
# [0,1,0,1,0,2,3]
# int main() {
#     int x = 10;
#     if(x > 5){
#         printf("Hello, World!");
#     }
# }
# """

# # Create a DataFrame
# df = pd.DataFrame({
#     'code': [code_snippet_1, code_snippet_2]
# })

# # Display the DataFrame
# df.head()


In [1]:
import code_tokenize
import re
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import pickle
import time
import os
import numpy as np
import pandas as pd

2024-07-17 16:10:51.635767: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 16:10:51.703029: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-17 16:10:51.706309: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-17 16:10:51.706320: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [ ]:
df = pd.read_csv("/home/irshad/Documents/Vuldetect_Source_code/diversevul_dataset/diversevul.csv")

In [ ]:
df.head()

In [ ]:
df.target.value_counts()

In [2]:
import re

def preprocessing(c_program):
    # Convert non-string inputs to string
    if not isinstance(c_program, str):
        c_program = str(c_program)
    
    # Apply preprocessing steps directly to the entire program string
    preprocessed_program = re.sub(r'\/\*[\s\S]*?\*\/|\/\/.*', '', c_program)  # Remove comments
    preprocessed_program = re.sub(r'#.*', '', preprocessed_program)            # Remove preprocessor directives
    preprocessed_program = preprocessed_program.replace('\\\n', '')             # Remove unnecessary backslashes
    preprocessed_program = preprocessed_program.replace('\t', '')              # Remove unnecessary indentation
    preprocessed_program = preprocessed_program.strip()                         # Remove leading/trailing whitespace
    return preprocessed_program

In [4]:
df['func']= df['func'].apply(preprocessing)

In [6]:
def my_tokenizer(text):
    tokens = code_tokenize.tokenize(text, lang="c", syntax_error="ignore")
    tokens_list = [str(x) for x in tokens]
    return tokens_list

In [ ]:
#df['code'] = df['code'].apply(my_tokenizer)

In [7]:
# tokenize the data that can be used by embeddings
#import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import pickle
import time


tokenizer_path = "/home/irshad/Documents/Vuldetect/tokenizer/tokenizer.pkl"
retrain = False
if os.path.exists(tokenizer_path) and not retrain:
    print("[INFO] Loading saved tokenizer")
    with open(tokenizer_path, "rb") as tokenizer_file:
        tokenizer = pickle.load(tokenizer_file)
else:
    t0 = time.time()
    print("[INFO] fitting tokenizer on TRAIN data...")
    tokenizer = Tokenizer(lower=False, analyzer = my_tokenizer)  # used in the research paper
    tokenizer.fit_on_texts(X_train.apply(lambda x: np.str_(x)))
    # save the tokenizer for future use
    with open(tokenizer_path, "wb") as tokenizer_file:
        pickle.dump(tokenizer, tokenizer_file, protocol=pickle.HIGHEST_PROTOCOL)
        # pickle.HIGHEST_PROTOCOL is highest protocol version available
    print(f"time taken to fit and save the tokenizer: {time()-t0} s")   

/home/irshad/anaconda3/envs/tensorflow_env/lib/python3.9/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


In [ ]:
#sequences = tokenizer.fit_on_texts_to_sequences(text)


In [8]:

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(df['code'])

# Get max training sequence length
maxlen = max([len(x) for x in train_sequences])

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding='post', truncating='post', maxlen=50)

# Output the results of our work
print("Word index:\n", word_index)
print('maximum length: ', maxlen)
print("\nTraining sequences:\n", train_sequences)
print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded.shape)
print("Training sequences data type:", type(train_sequences))
print("Padded Training sequences data type:", type(train_padded))

Word index:
 {';': 1, ',': 2, 'int': 3, '(': 4, ')': 5, '=': 6, '"': 7, '0': 8, '{': 9, '}': 10, 'x': 11, 'main': 12, 'a': 13, '10': 14, 'b': 15, 'printf': 16, '1': 17, '20': 18, 'Maximum is %d': 19, 'MAX': 20, 'return': 21, 'float': 22, '3.14': 23, 'y': 24, '42': 25, '[': 26, '2': 27, '3': 28, ']': 29, 'if': 30, '>': 31, '5': 32, 'Hello, World!': 33}
maximum length:  51

Training sequences:
 [[3, 12, 4, 5, 9, 3, 13, 6, 14, 1, 3, 15, 6, 18, 1, 16, 4, 7, 19, 7, 2, 20, 4, 13, 2, 15, 5, 5, 1, 21, 8, 1, 10], [22, 11, 6, 23, 1, 3, 24, 6, 25, 1, 26, 8, 2, 17, 2, 8, 2, 17, 2, 8, 2, 27, 2, 28, 29, 3, 12, 4, 5, 9, 3, 11, 6, 14, 1, 30, 4, 11, 31, 32, 5, 9, 16, 4, 7, 33, 7, 5, 1, 10, 10]]

Padded training sequences:
 [[ 3 12  4  5  9  3 13  6 14  1  3 15  6 18  1 16  4  7 19  7  2 20  4 13
   2 15  5  5  1 21  8  1 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [22 11  6 23  1  3 24  6 25  1 26  8  2 17  2  8  2 17  2  8  2 27  2 28
  29  3 12  4  5  9  3 11  6 14  1 30  4 11 31 32  5 

In [30]:
# Reverse the word index to get index -> word mapping
index_word = {index: word for word, index in word_index.items()}
tokens_list = [[index_word.get(idx, '[UNK]') for idx in seq] for seq in train_sequences]
# Print the tokens
for i, text_tokens in enumerate(tokens_list):
    print(text_tokens)

['int', 'main', '(', ')', '{', 'int', 'a', '=', '10', ';', 'int', 'b', '=', '20', ';', 'printf', '(', '"', 'Maximum is %d', '"', ',', 'MAX', '(', 'a', ',', 'b', ')', ')', ';', 'return', '0', ';', '}']
['float', 'x', '=', '3.14', ';', 'int', 'y', '=', '42', ';', '[', '0', ',', '1', ',', '0', ',', '1', ',', '0', ',', '2', ',', '3', ']', 'int', 'main', '(', ')', '{', 'int', 'x', '=', '10', ';', 'if', '(', 'x', '>', '5', ')', '{', 'printf', '(', '"', 'Hello, World!', '"', ')', ';', '}', '}']


['int', 'main', '(', ')', '{', 'int', 'a', '=', '10', ';', 'int', 'b', '=', '20', ';', 'printf', '(', '"', 'Maximum is %d', '"', ',', 'MAX', '(', 'a', ',', 'b', ')', ')', ';', 'return', '0', ';', '}']
['float', 'x', '=', '3.14', ';', 'int', 'y', '=', '42', ';', '[', '0', ',', '1', ',', '0', ',', '1', ',', '0', ',', '2', ',', '3', ']', 'int', 'main', '(', ')', '{', 'int', 'x', '=', '10', ';', 'if', '(', 'x', '>', '5', ')', '{', 'printf', '(', '"', 'Hello, World!', '"', ')', ';', '}', '}']


In [11]:
import gensim
from gensim.models import Word2Vec
word2vec_model=gensim.models.Word2Vec(
    sg=1,
    window=10,
    min_count=2,
    workers=16
)
word2vec_model.build_vocab(tokens_list)
start_time = time.time()
word2vec_model.train(sentences=tokens_list, total_examples = word2vec_model.corpus_count, epochs=word2vec_model.epochs)
print(f'Time taken : {(time.time() - start_time) / 60:.2f} mins')

In [28]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
print(num_cores)

In [25]:
model.wv.get_normed_vectors().shape

(17, 100)

In [26]:
y=word2vec_model.wv.index_to_key

In [27]:
y

[';',
 ',',
 '(',
 ')',
 'int',
 '=',
 '"',
 '0',
 'x',
 '{',
 '}',
 'printf',
 'b',
 '10',
 'a',
 'main',
 '1']

In [ ]:
word2vec_model.wv.most_similar('')

In [ ]:
word2vec_model.wv['printf']

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import pickle
import time


tokenizer_path = "/home/irshad/Documents/Vuldetect/tokenizer/tokenizer.pkl"
retrain = False
if os.path.exists(tokenizer_path) and not retrain:
    print("[INFO] Loading saved tokenizer")
    with open(tokenizer_path, "rb") as tokenizer_file:
        tokenizer = pickle.load(tokenizer_file)
else:
    t0 = time.time()
    print("[INFO] fitting tokenizer on TRAIN data...")
    tokenizer = Tokenizer(lower=False, analyzer = my_tokenizer)  # used in the research paper
    tokenizer.fit_on_texts(X_train.apply(lambda x: np.str_(x)))
    # save the tokenizer for future use
    with open(tokenizer_path, "wb") as tokenizer_file:
        pickle.dump(tokenizer, tokenizer_file, protocol=pickle.HIGHEST_PROTOCOL)
        # pickle.HIGHEST_PROTOCOL is highest protocol version available
    print(f"time taken to fit and save the tokenizer: {time()-t0} s")   